In [1]:
import sys
import os

In [2]:
sys.path.append(os.path.join(os.path.abspath(os.path.join('../..')), 'src'))

In [3]:
from importlib import reload

In [4]:
import pickle

In [5]:
import mysql_utils
import feature_extract

In [6]:
reload(build_graph)

NameError: name 'build_graph' is not defined

In [7]:
bows_path = os.path.join(os.path.abspath(os.path.join('../..')), 'src/data/sample_bows.pkl')

In [8]:
with open(bows_path, 'rb') as f:
    bows = pickle.load(f)

In [9]:
cvs = feature_extract.CountVecSimple()

In [10]:
cvs.fit(bows)

In [11]:
bow_mat = cvs.transform(bows)

In [12]:
bow_mat.shape

(2075, 115)

In [78]:
import numpy
import scipy
import pandas

In [13]:
pandas.Series(numpy.array(bow_mat.sum(axis=1)).reshape(bow_mat.shape[0],)).describe()

count    2075.000000
mean        2.685783
std         2.678095
min         1.000000
25%         1.000000
50%         2.000000
75%         3.000000
max        53.000000
dtype: float64

In [18]:
import metrics

In [21]:
reload(metrics)

<module 'metrics' from '/home/immersinn/gits/rssfeed_link_collector/src/metrics.py'>

In [73]:
bow_mat2 = bow_mat.tocsr()

In [81]:
oc = bow_mat2[orig_index,].copy()

In [82]:
oc.shape

(1, 115)

In [83]:
bow_mat.shape

(2075, 115)

In [22]:
dds = metrics.calcJMSDocScores(bow_mat)

In [23]:
dds.shape

(2075, 2075)

In [24]:
dds == dds.T

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ..., 
       [ True,  True,  True, ...,  True, False,  True],
       [ True,  True,  True, ..., False,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]], dtype=bool)

In [25]:
def print_top_scores(df, n=20):
    for i in df.index[:n]:
        print(df.title[i][:80] +\
              ''.join([' ' for _ in  range(max(0, -(len(df.title[i])-80)))]) +\
              "\t\t(score {:.2f})".format(df.score[i]))

### Simple JM Closeness Method

In [17]:
orig_index = cvs._bwm.lookup_docid('orig')

In [18]:
pandas.Series(dds[orig_index,:]).describe(percentiles=[0.5, 0.8, 0.9, 0.95, 0.99])

count    2075.000000
mean        0.033776
std         0.049774
min         0.000000
50%         0.008136
80%         0.065216
90%         0.094275
95%         0.118651
99%         0.190376
max         1.000000
dtype: float64

In [19]:
close_docs = numpy.argsort(dds[orig_index,:], )[-30:]
close_docs = numpy.where(dds[orig_index,:] >= 0.15)[0]
close_docs = [cvs._bwm.revlookup_docid(did) for did in close_docs]
print(len(close_docs))

48


In [20]:
scores = pandas.DataFrame([{'id' : did, 'score' : dds[orig_index, cvs._bwm.lookup_docid(did)]} \
                           for did in close_docs])
scores.index = scores.id
scores = scores[['score']]

In [23]:
jms_close = mysql_utils.query_docsDetails(close_docs, fields=['id', 'link', 'title', 'summary', 'published'])

In [24]:
jms_close.index = jms_close['id']
jms_close = jms_close[['link', 'title', 'summary', 'published']]
jms_close = jms_close.join(scores)
jms_close = jms_close.sort_values(by='score', ascending=False)

In [26]:
print_top_scores(jms_close, n=40)

FBI refused White House request to knock down recent Trump-Russia stories       		(score 1.00)
US officials corroborate aspects of Russia dossier                              		(score 0.42)
Trump aides spoke regularly to Russian officials during campaign                		(score 0.39)
Donald Trump Denounces ‘Un-American’ Intelligence Leaks to ‘Fake News’          		(score 0.34)
White House was warned Flynn could be blackmailed by Russia                     		(score 0.34)
CNN’s Acosta: Trump Only Calling on Conservative Media Can Only Be Described As 		(score 0.28)
Priebus: Take Trump Seriously When he Calls Media ‘The Enemy’                   		(score 0.24)
Dem Rep Moulton: If Trump Admin ‘Conspiring’ with Russia — That Is ‘The Definiti		(score 0.23)
Former Clinton aides apoplectic over Trump campaign-Russia reports              		(score 0.23)
Hollywood Reporter’s Wolff to CNN’s Stelter on Trump Coverage: Media Becoming ‘Q		(score 0.22)
NYT’s Friedman: Flynn Resignation Shows Russia Hac

### Simple Spectral 2-Way Method

In [27]:
import spectral_partition

In [ ]:
reload(spectral_partition)

In [28]:
dds_sym = (dds + dds.T) / 2

In [29]:
groups, counts, history = spectral_partition.spectralGraphPartition23(dds_sym, L=0.5, n_cutoff=15)

In [30]:
counts

{'threeway-pair': 0, 'twoway-pair': 2, 'twoway-single': 5}

In [31]:
orig_grp = numpy.where(groups == groups[orig_index])[0]
orig_grp = [cvs._bwm.revlookup_docid(did) for did in orig_grp]
_ = orig_grp.remove('orig')

In [32]:
len(orig_grp)

961

In [33]:
scores = [{'id': did, 'score' : dds_sym[orig_index, cvs._bwm.lookup_docid(did)]} for did in orig_grp]
scores = pandas.DataFrame(scores)
scores.index = scores['id']
scores = scores[['score']]

In [34]:
doc_info = mysql_utils.query_docsDetails(orig_grp, fields=['id', 'link', 'title', 'summary', 'published'])

In [35]:
doc_info.index = doc_info['id']
doc_info = doc_info[['link', 'title', 'summary', 'published']]
doc_info = doc_info.join(scores)
doc_info = doc_info.sort_values(by='score', ascending=False)

In [36]:
print_top_scores(doc_info, n=40)

FBI refused White House request to knock down recent Trump-Russia stories       		(score 1.00)
US officials corroborate aspects of Russia dossier                              		(score 0.42)
Trump aides spoke regularly to Russian officials during campaign                		(score 0.39)
Donald Trump Denounces ‘Un-American’ Intelligence Leaks to ‘Fake News’          		(score 0.34)
White House was warned Flynn could be blackmailed by Russia                     		(score 0.34)
CNN’s Acosta: Trump Only Calling on Conservative Media Can Only Be Described As 		(score 0.28)
Priebus: Take Trump Seriously When he Calls Media ‘The Enemy’                   		(score 0.24)
Dem Rep Moulton: If Trump Admin ‘Conspiring’ with Russia — That Is ‘The Definiti		(score 0.23)
Former Clinton aides apoplectic over Trump campaign-Russia reports              		(score 0.23)
Hollywood Reporter’s Wolff to CNN’s Stelter on Trump Coverage: Media Becoming ‘Q		(score 0.22)
KLEIN – Trump-Russia Claims Rethought: How the Oba

In [66]:
sum([i not in docs_l01 for i in jms_close.index]) / jms_close.shape[0]

0.375

In [68]:
test_size = 100
sum([i not in docs_l01 for i in doc_info.index[:test_size]]) / test_size

0.39